In [25]:
from datasets import load_dataset
from gensim.utils import tokenize
from tqdm import tqdm
from textblob import TextBlob
from collections import Counter
data=load_dataset('imdb')

In [2]:
all_sentences=[]
sent_threshold=32

for text_block in tqdm(data['train']['text']):
    for sentences in TextBlob(text_block).sentences:
        len_of_sent=sentences.words 
        for sent in sentences.split('.<br /><br />'):
            if len(len_of_sent)<sent_threshold:
                all_sentences.append(sent)
    
    


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:23<00:00, 1064.85it/s]


In [23]:
words=[]
for sent in tqdm(all_sentences):
    for word in tokenize(sent):
        words.append(word)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 227434/227434 [00:01<00:00, 147829.79it/s]


In [27]:
cntr=Counter(words)

In [55]:
word_threshold=0
our_words=set(['<unk>','<bos>','<eos>','<pad>'])
for word,cnt in cntr.items():
    if cnt<=word_threshold:
        our_words.add(word)

In [56]:
print(len(our_words))

28880
